# **💡0️⃣ [ 아래 코드를 실행하여 조별 번호를 입력하시오.]💡**

*   1조: 박유진, 김서연, 최진
*   2조: 서민경, 이유진, 최범영
*   3조: 정은주, 김장환, 권진경
*   4조: 정진교, 장채은, 조성환
*   5조: 전승재, 신은호, 김윤희
*   6조: 정현서, 진경은, 이정민
*   7조: 이성지, 김내경



In [2]:
print("[예]조의 번호를 입력하시오: 1")
team_idx = int(input(">> 조의 번호를 입력하시오: "))
print(f">> {team_idx}조 입니다.")
team_idx = 'team'+str(team_idx)

[예]조의 번호를 입력하시오: 1
>> 5조 입니다.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url, extract_archive
import random
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
from torchinfo import summary

URL = "https://github.com/JanghunHyeon/AISW4202-Project/releases/download/v.1.1.0/project_dataset.zip"
ROOT = "./content/data"
ZIP_PATH = os.path.join(ROOT, "project_dataset.zip")
OUT_DIR  = os.path.join(ROOT, "project_dataset")

os.makedirs(ROOT, exist_ok=True)
download_url(URL, root=ROOT, filename="project_dataset.zip")
extract_archive(ZIP_PATH, OUT_DIR)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Reproduce를 위한 Seed 고정
seed_id = 777
deterministic = True

random.seed(seed_id)
np.random.seed(seed_id)
torch.manual_seed(seed_id)
if device =='cuda':
    torch.cuda.manual_seed_all(seed_id)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

# **💡1️⃣ [ Design your custom model ]💡**
  ## 아래 코드를 수정하여 본인의 모델을 만들어 보세요.

In [ ]:
############################
# 1. 블록 정의
############################
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, c_in, c_out, stride=1):
        super().__init__()
        self.depth = nn.Conv2d(c_in, c_in, 3, stride, 1, groups=c_in, bias=False)
        self.point = nn.Conv2d(c_in, c_out, 1, bias=False)
        self.bn = nn.BatchNorm2d(c_out)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.bn(self.point(self.depth(x))))
        return x

class SEBlock(nn.Module):
    def __init__(self, ch, r=4):
        super().__init__()
        self.fc1 = nn.Linear(ch, ch // r, bias=False)
        self.fc2 = nn.Linear(ch // r, ch, bias=False)
        self.act = nn.ReLU()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        s = x.mean((2,3))                    # Squeeze
        s = self.sig(self.fc2(self.act(self.fc1(s))))
        s = s.view(s.size(0), -1, 1, 1)
        return x * s                         # Excitation

class DwResSEBlock(nn.Module):
    def __init__(self, c_in, c_out, stride):
        super().__init__()
        self.conv = DepthwiseSeparableConv(c_in, c_out, stride)
        self.se   = SEBlock(c_out)
        self.skip = nn.Sequential()
        if stride != 1 or c_in != c_out:
            self.skip = nn.Sequential(
                nn.Conv2d(c_in, c_out, 1, stride, bias=False),
                nn.BatchNorm2d(c_out))
        self.act = nn.ReLU()

    def forward(self, x):
        out = self.se(self.conv(x))
        out = out + self.skip(x)
        return self.act(out)

############################
# 2. MyCustomModel 교체
############################
class MyCustomModel(nn.Module):
    def __init__(self, num_classes=15):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU())

        self.stage1 = self._make_layer(32,  64,  2, stride=2) # 48→24
        self.stage2 = self._make_layer(64,  128, 2, stride=2) # 24→12
        self.stage3 = self._make_layer(128, 256, 3, stride=2) # 12→6

        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc   = nn.Linear(512, num_classes)

        self._initialize_weights()

    def _make_layer(self, c_in, c_out, blocks, stride):
        layers = [DwResSEBlock(c_in, c_out, stride)]
        layers += [DwResSEBlock(c_out, c_out, 1) for _ in range(blocks-1)]
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.pool(x).view(x.size(0), -1)
        return self.fc(x)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [5]:
model = MyCustomModel(num_classes=15).to(device)

inputs = torch.Tensor(1,3,48,48).to(device)
out = model(inputs)
print(out.shape)

AttributeError: 'MyCustomModel' object has no attribute 'stage4'

In [ ]:
print(out)

tensor([[ 3.8659e-02,  3.0929e-02,  3.9412e-02, -2.2221e-02,  1.1541e-02,
         -2.0074e-02, -3.9873e-02,  1.6007e-02,  3.4316e-02, -1.7259e-02,
         -1.1037e-02,  2.2265e-02,  2.9167e-02, -7.4874e-05,  1.5028e-02]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [8]:
summary(model, (1, 3, 48, 48))

Layer (type:depth-idx)                        Output Shape              Param #
MyCustomModel                                 [1, 15]                   --
├─Sequential: 1-1                             [1, 32, 48, 48]           --
│    └─Conv2d: 2-1                            [1, 32, 48, 48]           864
│    └─BatchNorm2d: 2-2                       [1, 32, 48, 48]           64
│    └─ReLU: 2-3                              [1, 32, 48, 48]           --
├─Sequential: 1-2                             [1, 64, 24, 24]           --
│    └─DwResSEBlock: 2-4                      [1, 64, 24, 24]           --
│    │    └─DepthwiseSeparableConv: 3-1       [1, 64, 24, 24]           2,464
│    │    └─SEBlock: 3-2                      [1, 64, 24, 24]           2,048
│    │    └─Sequential: 3-3                   [1, 64, 24, 24]           2,176
│    │    └─ReLU: 3-4                         [1, 64, 24, 24]           --
│    └─DwResSEBlock: 2-5                      [1, 64, 24, 24]           --
│    │    

# **💡2️⃣[ Dataset Processing ]💡**

## Dataset for Final Project
- Large-scale RGB image dataset
  - Training set: about 75,000 images
  - Validation set: 900 images
  
- Dataset specification
  - image size: 48 x 48
  - RGB scale image (3 channel)
  - 15 classes

In [9]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225]
    ),
])

data_dir = os.path.join(OUT_DIR, "data/ProjectDataset")
trainset = ImageFolder(os.path.join(data_dir, "train"), transform=transform_train)
valset = ImageFolder(os.path.join(data_dir, "val"), transform=transform_val)

trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=6)
valloader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=6)

# **💡3️⃣[ Train your custom model ]💡**


In [10]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [11]:
print(len(trainloader))
epochs = 50

for epoch in range(epochs):  # loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 iterations
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

    lr_sche.step()

    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)

            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 1,200 validation images: %.3f %%' % (100 * correct / total))


print('Finished Training')

288
[1,    50] loss: 2.058
[1,   100] loss: 1.671
[1,   150] loss: 1.505
[1,   200] loss: 1.407
[1,   250] loss: 1.317
Accuracy of the network on the 1,200 validation images: 6.667 %
[2,    50] loss: 1.184
[2,   100] loss: 1.150
[2,   150] loss: 1.134
[2,   200] loss: 1.103
[2,   250] loss: 1.065
Accuracy of the network on the 1,200 validation images: 6.667 %
[3,    50] loss: 0.987
[3,   100] loss: 0.970
[3,   150] loss: 0.950
[3,   200] loss: 0.956
[3,   250] loss: 0.930
Accuracy of the network on the 1,200 validation images: 6.667 %
[4,    50] loss: 0.846
[4,   100] loss: 0.860
[4,   150] loss: 0.846
[4,   200] loss: 0.857
[4,   250] loss: 0.859
Accuracy of the network on the 1,200 validation images: 6.667 %
[5,    50] loss: 0.793
[5,   100] loss: 0.798


KeyboardInterrupt: 

# 💡4️⃣ [Model Evaluation]💡
* Validation Set을 이용한 성능 검증

In [15]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)

        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 1,200 validation images: %.3f %%' % (100 * correct / total))

Accuracy of the network on the 1,200 validation images: 6.667 %


# 💡5️⃣ [Save Model]💡
1. 아래 코드를 동작하면 Google 인증 팝업이 뜨고 인증을 요청.
2. 인증후, Drive에 Final_Project/Model이라는 폴더가 생성됨.
3. 생성된 폴더 안에 **model_{team_idx}.pt** 라는 파일이 생성됨.
4. 생성된 파일과 코드를 다운받아 LMS에 제출




In [16]:
fname = f"model_{team_idx}.pt"

model.eval()
example_input = torch.randn(1,3,48,48).to(device)
traced_script = torch.jit.trace(model, example_input)
traced_script.save(fname)


# 구축한 데이터셋 Google Drive로 저장
# #from google.colab import drive
# import shutil

# # 1) Drive 마운트
# #drive.mount('/content/drive')
# data_dir = './content'
# out_dirs = './content/Final_Project/model'
# os.makedirs(out_dirs, exist_ok=True)

# shutil.copy(os.path.join(data_dir, fname), os.path.join(out_dirs, fname))
# print("Copied to Drive → MyDrive")